In [1]:
import pandas as pd

'c:\\Users\\Micha\\Github Repo'

In [2]:
df = pd.read_excel(r"C:\Users\Micha\OneDrive - Høyskolen Kristiania\MsC Business Analytics\Master Thesis\Python\Subject_info.xlsx")

In [22]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 178 entries, 0 to 177
Data columns (total 10 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Subject            178 non-null    object 
 1   Sex                178 non-null    object 
 2   Weight             178 non-null    float64
 3   Research Group     178 non-null    object 
 4   APOE A1            176 non-null    float64
 5   APOE A2            176 non-null    float64
 6   Age                178 non-null    float64
 7   dataset_split      178 non-null    object 
 8   File_Path          178 non-null    object 
 9   File_Path_desktop  178 non-null    object 
dtypes: float64(4), object(6)
memory usage: 14.0+ KB


In [33]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = Baseline3DCNN().to(device)


In [36]:
# Initialize, train, validate, and test the model
model = Baseline3DCNN(**params).to(device)
train_loss = train_model(model, train_loader, criterion, optimizer, num_epochs=10)
val_accuracy = validate_model(model, val_loader, criterion)
test_accuracy = test_model(model, test_loader)

# Log the experiment
log_experiment(params, train_loss, val_accuracy, test_accuracy)

NameError: name 'train_loader' is not defined

In [5]:
import pandas as pd
df = pd.read_excel(r"C:\Users\Micha\OneDrive - Høyskolen Kristiania\MsC Business Analytics\Master Thesis\Python\Subject_info.xlsx")

In [6]:
# Assuming df is your DataFrame
def preprocess_labels(df):
    # Mapping similar categories to a single category
    label_mapping = {
        'EMCI': 'MCI',
        'LMCI': 'MCI',
        'SMC': 'CN'  # If you want SMC to be considered as CN, include this; remove if not
    }
    df['Research Group'] = df['Research Group'].replace(label_mapping)
    return df

preprocess_labels(df)

,Subject,Sex,Weight,Research Group,APOE A1,APOE A2,Age,dataset_split,File_Path,File_Path_desktop
0,002_S_0295,M,73.0,CN,3.0,4.0,90.0,train,C:\Users\Micha\OneDrive - Høyskolen Kristiania...,D:\Data\Preprocessed\Fused Images\002_S_0295_f...
1,002_S_0413,F,57.6,CN,3.0,3.0,81.5,test,C:\Users\Micha\OneDrive - Høyskolen Kristiania...,D:\Data\Preprocessed\Fused Images\002_S_0413_f...
2,002_S_0685,F,68.9,CN,3.0,3.0,95.8,test,C:\Users\Micha\OneDrive - Høyskolen Kristiania...,D:\Data\Preprocessed\Fused Images\002_S_0685_f...
3,002_S_0729,F,65.8,MCI,3.0,4.0,71.3,validation,C:\Users\Micha\OneDrive - Høyskolen Kristiania...,D:\Data\Preprocessed\Fused Images\002_S_0729_f...
4,002_S_1155,M,64.9,MCI,3.0,3.0,64.0,train,C:\Users\Micha\OneDrive - Høyskolen Kristiania...,D:\Data\Preprocessed\Fused Images\002_S_1155_f...
...,...,...,...,...,...,...,...,...,...,...
173,941_S_4377,F,121.6,MCI,3.0,4.0,69.5,test,C:\Users\Micha\OneDrive - Høyskolen Kristiania...,D:\Data\Preprocessed\Fused Images\941_S_4377_f...
174,941_S_4420,M,79.4,MCI,3.0,3.0,81.5,train,C:\Users\Micha\OneDrive - Høyskolen Kristiania...,D:\Data\Preprocessed\Fused Images\941_S_4420_f...
175,941_S_4764,F,77.6,MCI,3.0,3.0,82.8,train,C:\Users\Micha\OneDrive - Høyskolen Kristiania...,D:\Data\Preprocessed\Fused Images\941_S_4764_f...
176,941_S_5124,F,78.9,CN,3.0,3.0,76.8,test,C:\Users\Micha\OneDrive - Høyskolen Kristiania...,D:\Data\Preprocessed\Fused Images\941_S_5124_f...


In [19]:

import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import numpy as np
import nibabel as nib
from torchvision import transforms
from tqdm import tqdm
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader

In [20]:
class NiiDataset(Dataset):
    def __init__(self, df, transform=None):
        self.paths = df['File_Path_desktop'].tolist()
        self.labels = pd.Categorical(df['Research Group']).codes
        self.subjects = df['Subject'].tolist()  # Assuming 'Subject' is the column name for subject identifiers
        self.transform = transform

    def __len__(self):
        return len(self.paths)

    def __getitem__(self, idx):
        path = self.paths[idx]
        image = nib.load(path).get_fdata()
        image = np.expand_dims(image, axis=0)
        if self.transform:
            image = self.transform(image)
        label = torch.tensor(self.labels[idx], dtype=torch.long)
        subject = self.subjects[idx]  # Get the subject identifier
        
        return image, label, path, subject


In [21]:

def load_datasets(df):
    # Define custom transform functions here
    def custom_transform(image):
        # Convert to tensor
        tensor_image = torch.from_numpy(image).float()
        # Normalize
        tensor_image = (tensor_image - tensor_image.min()) / (tensor_image.max() - tensor_image.min()) * (1.0 - 0.0) + 0.0
        return tensor_image
    

    transform = custom_transform
    
    train_df = df[df['dataset_split'] == 'train']
    val_df = df[df['dataset_split'] == 'validation']
    test_df = df[df['dataset_split'] == 'test']
    
    train_dataset = NiiDataset(train_df, transform=transform)
    val_dataset = NiiDataset(val_df, transform=transform)
    test_dataset = NiiDataset(test_df, transform=transform)
    
    return train_dataset, val_dataset, test_dataset

In [10]:
def create_dataloaders(train_dataset, val_dataset, test_dataset, batch_size=4):
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
    test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)
    return train_loader, val_loader, test_loader

In [11]:

# Model definition
class Baseline3DCNN(nn.Module):
    def __init__(self, num_classes=3, init_filters=32, kernel_size=3, stride=2, num_fc_units=128):
        super(Baseline3DCNN, self).__init__()
        self.conv1 = nn.Conv3d(1, init_filters, kernel_size=kernel_size, stride=stride, padding=1)
        self.conv2 = nn.Conv3d(init_filters, init_filters*2, kernel_size=kernel_size, stride=stride, padding=1)
        self.conv3 = nn.Conv3d(init_filters*2, init_filters*4, kernel_size=kernel_size, stride=stride, padding=1)
        self.pool = nn.MaxPool3d(2)
        self.relu = nn.ReLU()

        # Compute the flattened size after all convolutions and pooling
        self.final_dim = self._get_conv_output_dim(193, 3, stride, kernel_size, init_filters*4)
        self.fc1 = nn.Linear(self.final_dim, num_fc_units)
        self.fc2 = nn.Linear(num_fc_units, num_classes)

    def _get_conv_output_dim(self, input_dim, num_convs, stride, kernel_size, num_filters):
        output_dim = input_dim
        for _ in range(num_convs):
            output_dim = ((output_dim - kernel_size + 2 * (kernel_size // 2)) // stride + 1) // 2  # Pooling divides size by 2
        return output_dim * output_dim * output_dim * num_filters

    def forward(self, x):
        x = self.pool(self.relu(self.conv1(x)))
        x = self.pool(self.relu(self.conv2(x)))
        x = self.pool(self.relu(self.conv3(x)))
        x = torch.flatten(x, 1)
        x = self.relu(self.fc1(x))
        x = self.fc2(x)
        return x

In [12]:
import torch

# Check if CUDA is available and set the device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")


Using device: cpu


In [13]:
# Create a mapping of label codes to label names
unique_labels = pd.Categorical(df['Research Group'])
label_mapping = {code: name for code, name in zip(unique_labels.codes, unique_labels.categories)}


In [16]:
def train_model(model, train_loader, criterion, optimizer, num_epochs=10, device='cpu'):
    model.to(device)  # Move the model to the appropriate device
    model.train()
    train_results = []
    for epoch in range(num_epochs):
        for images, labels, paths, subjects in tqdm(train_loader, desc=f'Epoch {epoch+1}/{num_epochs}'):
            images, labels = images.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            _, predicted_indices = torch.max(outputs.data, 1)
            
            # Debugging code to inspect predicted indices
            print("Predicted Indices:", predicted_indices.cpu().numpy())
            
            # Check for any predicted indices not present in label_mapping
            for code in predicted_indices.cpu().numpy():
                if code not in label_mapping:
                    print(f"Predicted index {code} not found in label_mapping!")
            
            # Original code to retrieve predicted labels
            predicted_labels = [label_mapping[code] for code in predicted_indices.cpu().numpy()]
            
            # Continue with original code
            for label, pred, path, subject in zip(labels.cpu().numpy(), predicted_labels, paths, subjects):
                train_results.append({
                    'Subject': subject,
                    'Path': path,
                    'Actual Label': label_mapping[label.item()],  # Now using label_mapping to get actual label name
                    'Prediction': pred,
                    'Type': 'Train'
                })
    return train_results

def validate_model(model, val_loader, criterion, device='cpu'):
    model.to(device)
    model.eval()
    validation_results = []
    with torch.no_grad():
        for images, labels, paths, subjects in tqdm(val_loader, desc='Validation'):
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted_indices = torch.max(outputs.data, 1)
            predicted_labels = [label_mapping[code] for code in predicted_indices.cpu().numpy()]
            for label, pred, path, subject in zip(labels.cpu().numpy(), predicted_labels, paths, subjects):
                validation_results.append({
                    'Subject': subject,
                    'Path': path,
                    'Actual Label': label_mapping[label.item()],
                    'Prediction': pred,
                    'Type': 'Validation'
                })
    return validation_results

def test_model(model, test_loader, device='cpu'):
    model.to(device)
    model.eval()
    test_results = []
    with torch.no_grad():
        for images, labels, paths, subjects in tqdm(test_loader, desc='Testing'):
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted_indices = torch.max(outputs.data, 1)
            predicted_labels = [label_mapping[code] for code in predicted_indices.cpu().numpy()]
            for label, pred, path, subject in zip(labels.cpu().numpy(), predicted_labels, paths, subjects):
                test_results.append({
                    'Subject': subject,
                    'Path': path,
                    'Actual Label': label_mapping[label.item()],
                    'Prediction': pred,
                    'Type': 'Test'
                })
    return test_results


In [17]:
train_dataset, val_dataset, test_dataset = load_datasets(df)

In [18]:
# Configuration
config = {
    'num_classes': 3,
    'init_filters': 32,
    'kernel_size': 3,
    'stride': 2,
    'num_fc_units': 128,
    'optimizer': 'Adam',
    'loss_criterion': 'CrossEntropyLoss',
    'num_epochs': 10
}
config_df = pd.DataFrame([config])

# Model Initialization
model = Baseline3DCNN(num_classes=config['num_classes'],
                      init_filters=config['init_filters'],
                      kernel_size=config['kernel_size'],
                      stride=config['stride'],
                      num_fc_units=config['num_fc_units']).to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters())

# Prepare DataLoaders (assuming your datasets are already split and loaded into variables train_dataset, val_dataset, test_dataset)
train_loader = DataLoader(train_dataset, batch_size=4, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=4, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=4, shuffle=False)

# Training, Validation, Testing
train_results = train_model(model, train_loader, criterion, optimizer, num_epochs=config['num_epochs'], device=device)
validate_results = validate_model(model, val_loader, criterion, device=device)
test_results = test_model(model, test_loader, device=device)

# Combine results from all phases
all_results = pd.DataFrame(train_results + validate_results + test_results)

import datetime

# Assuming config_df and all_results are defined elsewhere in your script
# Get the current date and time
current_time = datetime.datetime.now()
formatted_time = current_time.strftime('%Y-%m-%d_%H-%M-%S')  # Format as Year-Month-Day_Hour-Minute-Second

# Create a unique filename for each experiment run
filename = f'{formatted_time}_Experiment.xlsx'

# Export to Excel with a unique filename based on the current date and time
with pd.ExcelWriter(filename) as writer:
    config_df.to_excel(writer, sheet_name='Configuration')
    all_results.to_excel(writer, sheet_name='Results')


Epoch 1/10:   0%|          | 0/27 [00:00<?, ?it/s]

Epoch 1/10:   0%|          | 0/27 [00:00<?, ?it/s]


FileNotFoundError: No such file or no access: 'D:/Data/Preprocessed/Fused Images/032_S_0677_fused.nii'

In [63]:
# Assuming df is your DataFrame containing the data
unique_labels = pd.Categorical(df['Research Group'])
label_codes = unique_labels.codes
label_names = unique_labels.categories

# Create the mapping from codes to names
label_mapping = {code: name for code, name in zip(range(len(label_names)), label_names)}
print("Label Mapping:", label_mapping)


Label Mapping: {0: 'AD', 1: 'CN', 2: 'MCI'}


In [17]:
def evaluate_model(model, loader, criterion, phase="Validation"):
    model.eval()
    predictions = []
    true_labels = []
    image_paths = []
    losses = []
    softmax = nn.Softmax(dim=1)

    with torch.no_grad():
        for images, labels in loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs, 1)
            probs = softmax(outputs)

            predictions.extend(predicted.cpu().numpy())
            true_labels.extend(labels.cpu().numpy())
            losses.append(criterion(outputs, labels).item())

            # Assuming you have a way to access paths here
            image_paths.extend(loader.dataset.paths)

    accuracy = 100 * (np.array(predictions) == np.array(true_labels)).sum() / len(predictions)
    print(f'{phase} Accuracy: {accuracy}%')

    return predictions, true_labels, image_paths, losses, accuracy


In [18]:


def compute_metrics(predictions, labels):
    # Accuracy
    accuracy = accuracy_score(labels, predictions)
    # Sensitivity (Recall)
    sensitivity = recall_score(labels, predictions, average='macro')  # Adjust average as needed
    # Specificity - Calculate from confusion matrix
    tn, fp, fn, tp = confusion_matrix(labels, predictions).ravel()
    specificity = tn / (tn + fp)

    return {
        "accuracy": accuracy,
        "sensitivity": sensitivity,
        "specificity": specificity
    }
    


In [8]:
!pip install nibabel

In [19]:
train_dataset, val_dataset, test_dataset = load_datasets(df)
train_loader, val_loader, test_loader = create_dataloaders(train_dataset, val_dataset, test_dataset)


# Initialize model, loss criterion, and optimizer
model = Baseline3DCNN(**params).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# Evaluate the model on validation and test sets
val_predictions, val_labels, val_paths, val_losses, val_accuracy = evaluate_model(model, val_loader, criterion, "Validation")
test_predictions, test_labels, test_paths, test_losses, test_accuracy = evaluate_model(model, test_loader, criterion, "Testing")

# Compute metrics
val_metrics = compute_metrics(val_predictions, val_labels)
test_metrics = compute_metrics(test_predictions, test_labels)

# Log results to a DataFrame and save to Excel
results_df = pd.DataFrame({
    'Path': val_paths + test_paths,
    'True Label': val_labels + test_labels,
    'Predicted Label': val_predictions + test_predictions,
    'Phase': ['Validation']*len(val_paths) + ['Testing']*len(test_paths),
    'Loss': val_losses + test_losses
})
results_df.to_excel("model_results.xlsx", index=False)

# Print metrics for review
print("Validation Metrics:", val_metrics)
print("Test Metrics:", test_metrics)

Validation Accuracy: 5.555555555555555%
Testing Accuracy: 5.555555555555555%


NameError: name 'accuracy_score' is not defined

In [ ]:

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
params = {
    'num_classes': 3,
    'init_filters': 32,
    'kernel_size': 3,
    'stride': 2,
    'num_fc_units': 128
}
model = Baseline3DCNN(**params).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

train_loss = train_model(model, train_loader, criterion, optimizer, num_epochs=10)
val_accuracy = validate_model(model, val_loader, criterion)
test_accuracy = test_model(model, test_loader)